# HW8 (20')

<font size='4'>

For this assignment, it is a combination of jupyter notebook assignment and python scripts.

For Q1, please upload your outputs including codes and graphics to your own GitHub repository. <br> You will need to disclose your GitHub repository below.

For Q2, please submit this jupyter notebook as an HTML or PDF file.

First of all, print your name (First and Last) below.

Sofia Charlot

## 0. Import relevant packages

In [18]:
import os
import numpy as np
import pandas as pd
import scipy.stats as stats
import scipy.io as sio
import statsmodels.formula.api as smf

## Q1. Convert your HW7 to python scripts. (10')

<font size='4'>

- Under your working directory, there should be a folder called `self_py_fun`.
- Create a new python file called `HW8Fun.py` and move previously defined functions `produce_trun_mean_cov()`, `plot_trunc_mean()`, and `plot_trunc_cov()` to that file. Make sure you import proper packages.
- Create another main file `HW8_main.py`.
- Import relevant packages, modules, and/or function.
- Copy the global variables and call your functions inside `HW8_main.py`.
- A major difference compared to HW7 is that you are asked to save those figures to your local working environment.
    - Create a new directory `K114` under your current working directory.
    - For mean functions, please save it as a `Mean.png` output using `plt.savefig()` function.
    - The changes should be made within `HW8Fun.py` rather than `HW8_main.py`.
    - For covariance matrices, please save them as `Covariance_Target.png`, `Covariance_Non-Target.png`, and `Covariance_All.png` outputs using the same function above.
    - To summarize, there should be **four** figures under `K114` folder.
- Upload your entire work to your GitHub repository via push button.

In [90]:
# Provide your GitHub repository link below in the Markdown chunk. Remember to make it public and make the link clickable.
# Do not include sensitive information in your GitHub repository.

https://github.com/S-charlo/BIOS-584/blob/main/HW8.ipynb

In [3]:
!python HW8_main.py

All figures saved to K114 folder.


## Q2. A real-world data anlaysis using `Pandas` and `Scipy` (10')

<font size='4'>

- Back to the `PTSD dataset.xlsx`, let's import the dataset and name it `ptsd_df`. (no point since everyone has done it a couple of times before.)

In [4]:
# Write your own code
ptsd_df = pd.read_excel("data/PTSD dataset.xlsx", sheet_name="main_dataset")

### Q2.1. Univariate comparison (3')

<font size='4'> 
    
- Suppose that we would like to examine the utility/effect of an intervention program for patients with PTSD.
- We measure PCL5 scores at completion (`pcl5week_score.completion`) and PCL5 score at 3-month follow-up (`pcl5month_score.3_month_follow_up`). Let's assume the first score is pre-intervention and the second score is post-intervention.
- Report the summary statistics for each variable including mean, std, median, Q1, and Q3.
- Note that each patient will receive such two PCL5 scores. Use a appropriate statistical test to perform the univariate comparison. Report the outputing statistic and p-value.
- Before you run the statistic test, determine the data type and check the missingness of two columns. In particular, report the number of NA values for each variable.

In [12]:
# Write your own code
summary_stats = ptsd_df[['pcl5week_score.completion', 'pcl5month_score.3_month_follow_up']].agg(
    ['mean', 'std', 'median', 
     lambda x: np.percentile(x.dropna(), 25),
     lambda x: np.percentile(x.dropna(), 75)]
).T

summary_stats.columns = ['Mean', 'SD', 'Median', 'Q1', 'Q3']
print("\nSummary Statistics:\n", summary_stats)

print(ptsd_df.dtypes)

print("\nMissing values per variable:")
print(ptsd_df[['pcl5week_score.completion', 'pcl5month_score.3_month_follow_up']].isna().sum())

diff = ptsd_df['pcl5month_score.3_month_follow_up'] - ptsd_df['pcl5week_score.completion']
shapiro_test = stats.shapiro(diff.dropna())

print(f"Shapiro-Wilk test statistic: {shapiro_test.statistic:.4f}, p-value: {shapiro_test.pvalue:.4f}")

w_stat, p_val = stats.wilcoxon(
    ptsd_df['pcl5month_score.3_month_follow_up'],
    ptsd_df['pcl5week_score.completion'],
    nan_policy='omit'
)
print("\nWilcoxon Signed-Rank Test Results:")
print(f"Statistic = {w_stat:.4f}, p-value = {p_val:.4f}")


Summary Statistics:
                                         Mean         SD  Median    Q1     Q3
pcl5week_score.completion          29.438596  19.373840    25.0  15.0  42.00
pcl5month_score.3_month_follow_up  31.719828  18.812908    29.5  16.0  44.25
record_id               int64
ptsdpresent_caps        int64
caps_minuspcl           int64
caps_minuspcl_code      int64
caps_intake             int64
                       ...   
marines               float64
navy                  float64
coastguard            float64
nationalguard         float64
reserve               float64
Length: 439, dtype: object

Missing values per variable:
pcl5week_score.completion             27
pcl5month_score.3_month_follow_up    251
dtype: int64
Shapiro-Wilk test statistic: 0.9794, p-value: 0.0024

Wilcoxon Signed-Rank Test Results:
Statistic = 7509.0000, p-value = 0.0000


### Q2.2. Multiple Linear Regression (7')

<font size='4'>

- Select columns specified in the following code chunk and create a subset dataset named `ptsd_sub_df`.
- Fit a linear regression to examine the association between `caps_intake` (continuous outcome) and the remaining covariates (as predictors) using `ptsd_sub_df`.
    - Note that all covariates ending with `_code` are categorical variables.
- Use the instruction here to write the formula for linear regression in Python.
    - https://www.statsmodels.org/stable/example_formulas.html
- Report the output page including R2, adjusted R2, and parameter estimates, SE, 95% confidence intervals, and p-values.
- Provide a brief interpretation for all significant predictors (p<0.05) excluding the intercept.
- Relevant label information includes:
    - `employment_code`: 1: Employed, 2: Unemployed, 3: Retired, 4: Disabled/Unable to work, 5: Student, 6: Other.
    - `rank_code`: 1. Enlisted, 2: Officer, 3: Other

In [14]:
# The following column names are used for linear regression.
# Do not delete.
relevant_col_names = ['caps_intake', 'age_iop', 'gender_code', 'sexualorient_code', 'race_code', 'ethnicity_code', 
                      'education_code', 'employment_code',
                      'rank_code', 'branch_code', 'mdd_code', 'ctq_total_score', 'sexual_trauma', 'sud_code']

In [17]:
# Write your own code
ptsd_sub_df = ptsd_df[relevant_col_names].copy()

formula = 'caps_intake ~ age_iop + ctq_total_score + sexual_trauma + C(gender_code) + C(race_code) + C(ethnicity_code) + C(education_code) + C(rank_code) + C(branch_code) + C(sud_code) +  C(sexualorient_code) + C(employment_code) + C(mdd_code)'

model = smf.ols(formula=formula, data=ptsd_sub_df).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            caps_intake   R-squared:                       0.114
Model:                            OLS   Adj. R-squared:                  0.051
Method:                 Least Squares   F-statistic:                     1.796
Date:                Fri, 07 Nov 2025   Prob (F-statistic):            0.00697
Time:                        03:44:28   Log-Likelihood:                -1569.0
No. Observations:                 448   AIC:                             3200.
Df Residuals:                     417   BIC:                             3327.
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

# Write your interpretations below:
Participants in rank 2 had caps intake scores 3.06 points lower than the refernce group (p = 0.022) meaning that participants in rank 2 had significantly less PTSD symptoms at intake.

Participants in employment group 4 had caps intake scores 2.07 points higher than the refernce group (p = 0.029) meaning that participants in employment group 4 had significantly greater PTSD symptom severity at intake.

Each one point increase in the CTQ total score was associated with a 0.04 point increase in caps intake score (p =0.035). This means that there is a positive association between childhood trauma severity and PTSD symptoms.
